In [82]:
import fastf1
import pandas as pd
import numpy as np

In [65]:
session = fastf1.get_session(year=2024, identifier='Race', gp='Spain')
session.load(laps=True, telemetry=True, weather=False, messages=False)

core           INFO 	Loading data for Spanish Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '44', '63', '16', '55', '81', '11', '10', '31', '27', '14', '24', '18', '3', '77', '20', '23', '22', '2']


In [87]:
LEC_LAPS = session.laps.pick_driver('LEC').pick_laps(9)
VER_LAPS = session.laps.pick_driver('VER').pick_laps(7)

reference = VER_LAPS

In [100]:
from datetime import timedelta
from pandas import timedelta_range


ver_telemetry = VER_LAPS.get_telemetry()
lec_telemetry = LEC_LAPS.get_telemetry()

# reference = ver_telemetry.set_index('Time')
# lec_reindexed = lec_telemetry.set_index('Time').reindex(timedelta_range(start=timedelta(seconds=0), end=lec_telemetry.index.max()))
# resampled = reference.resample(rule='200ms', closed='left')[['Speed', 'Throttle']].mean()
# lec_reindexed['Speed'] = resampled['Speed']
# lec_reindexed['Throttle'] = resampled['Throttle']
# lec_reindexed
ver_telemetry.Speed/3600*1000

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


2      76.666667
3      76.944444
4      77.222222
5      77.222222
6      77.500000
         ...    
606    75.277778
607    75.555556
608    75.833333
609    76.111111
610    76.111111
Name: Speed, Length: 609, dtype: float64

session.laps

In [ ]:
formatted_laps = session.laps[[
                "Driver",
                "Team",
                "LapNumber",
                "LapTime",
                "Sector1Time",
                "Sector2Time",
                "Sector3Time",
                "SpeedI1",
                "SpeedI2",
                "SpeedFL",
                "Stint",
                "TyreLife",
                "Position",
                "Compound",
]]

formatted_laps.rename(columns={"SpeedI1": "ST1", "SpeedI2": "ST2", "SpeedFL": "ST3"}, inplace=True)
formatted_laps["LapTime"].round(3)
formatted_laps["Sector1Time"].round(3)
formatted_laps["Sector2Time"].round(3)
formatted_laps["Sector3Time"].round(3)
formatted_laps.assign(
    IsBestS1=lambda x: (formatted_laps['Sector1Time'].min() == x['Sector1Time']),
    IsBestS2=lambda x: (formatted_laps['Sector2Time'].min() == x['Sector2Time']),    
    IsBestS3=lambda x: (formatted_laps['Sector3Time'].min() == x['Sector3Time'])    
)

# formatted_laps.assign(
#     IsBestS1=lambda x: (
#         formatted_laps.groupby('Driver', as_index=True)["Sector1Time"].min() == x.groupby('Driver', as_index=True)["Sector1Time"] 
#     )
# )

# PBs = formatted_laps.groupby('Driver')['LapTime'].min()
# PBs.index

# formatted_laps.assign(
#     IsBestS1=lambda x: (
#         x['LapTime'] == PBs[x['Driver']] 
#     )
# )

# formatted_laps.groupby('Driver')['LapTime'].aggregate('min')

# for driver, laptime in PBs.items():
#     laptimes = formatted_laps[formatted_laps.Driver == driver].LapTime 
#     formatted_laps.loc[formatted_laps.Driver == driver, 'IsPB'] = laptimes == laptime 

# formatted_laps[formatted_laps.Driver == 'VER']

formatted_laps[formatted_laps.Driver == 'VER']


/var/folders/0h/_ft064n17r98nzj1w0nz7dq00000gn/T/ipykernel_2755/1513574309.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_laps.rename(columns={"SpeedI1": "ST1", "SpeedI2": "ST2", "SpeedFL": "ST3"}, inplace=True)


,Driver,Team,LapNumber,LapTime,Sector1Time,Sector2Time,Sector3Time,ST1,ST2,ST3,IsPersonalBest,Stint,TyreLife,Position,Compound
0,VER,Red Bull Racing,1.0,0 days 00:01:17.639000,NaT,0 days 00:00:31.613000,0 days 00:00:25.440000,292.0,233.0,109.0,False,1.0,1.0,1.0,MEDIUM
1,VER,Red Bull Racing,2.0,0 days 00:01:55.479000,0 days 00:00:31.698000,0 days 00:00:46.293000,0 days 00:00:37.488000,176.0,184.0,NaN,False,1.0,2.0,1.0,MEDIUM
2,VER,Red Bull Racing,3.0,0 days 00:02:04.721000,0 days 00:00:37.877000,0 days 00:00:46.298000,0 days 00:00:40.546000,191.0,61.0,270.0,False,2.0,3.0,1.0,MEDIUM
3,VER,Red Bull Racing,4.0,0 days 00:01:09.691000,0 days 00:00:17.618000,0 days 00:00:30.970000,0 days 00:00:21.103000,293.0,230.0,270.0,True,2.0,4.0,1.0,MEDIUM
4,VER,Red Bull Racing,5.0,0 days 00:01:10.026000,0 days 00:00:17.716000,0 days 00:00:31.158000,0 days 00:00:21.152000,285.0,230.0,270.0,False,2.0,5.0,1.0,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,VER,Red Bull Racing,67.0,0 days 00:01:08.854000,0 days 00:00:17.289000,0 days 00:00:30.872000,0 days 00:00:20.693000,298.0,237.0,277.0,False,4.0,18.0,1.0,MEDIUM
67,VER,Red Bull Racing,68.0,0 days 00:01:09.216000,0 days 00:00:17.653000,0 days 00:00:30.863000,0 days 00:00:20.700000,297.0,235.0,277.0,False,4.0,19.0,1.0,MEDIUM
68,VER,Red Bull Racing,69.0,0 days 00:01:12.228000,0 days 00:00:17.294000,0 days 00:00:30.769000,0 days 00:00:24.165000,300.0,237.0,NaN,False,4.0,20.0,1.0,MEDIUM
69,VER,Red Bull Racing,70.0,0 days 00:01:27.830000,0 days 00:00:34.138000,0 days 00:00:31.932000,0 days 00:00:21.760000,293.0,220.0,278.0,False,5.0,4.0,1.0,SOFT


In [8]:
lec_telemetry = laps.pick_driver('16').pick_lap(20).get_telemetry()
ver_telemetry = laps.pick_driver('4').pick_lap(20).get_telemetry()

filter_columns = ['Time', 'Speed', 'Distance', 'RelativeDistance']

lec_telemetry = lec_telemetry[filter_columns]
ver_telemetry = ver_telemetry[filter_columns]

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


In [18]:
def get_interpolated_time_at_distance(telemetry, relative_distance_point: float | int) -> float:
    threshold_index = 0
    
    RELATIVE_DISTANCE_COLUMN_INDEX = 3
    TIME_INDEX = 0

    TOLERANCE = 0.0005

    for index, row in enumerate(telemetry.values):
        if row[RELATIVE_DISTANCE_COLUMN_INDEX] <= relative_distance_point:
            threshold_index = index
        else: 
            break
    
    if np.abs(telemetry.values[threshold_index][RELATIVE_DISTANCE_COLUMN_INDEX]-relative_distance_point) < TOLERANCE: 
        return telemetry.values[threshold_index][TIME_INDEX]

    lower_than_avg_data_row = telemetry.values[threshold_index]
    higher_than_avg_data_row = telemetry.values[threshold_index + 1]
    
    lo_time = lower_than_avg_data_row[TIME_INDEX]
    hi_time = higher_than_avg_data_row[TIME_INDEX]
    
    lo_distance = lower_than_avg_data_row[3]
    hi_distance = higher_than_avg_data_row[3]
    
    diff = hi_distance - lo_distance

    lo_diff = relative_distance_point - lo_distance
    hi_diff = hi_distance - relative_distance_point

    return (lo_diff / diff) * lo_time + (hi_diff / diff) * hi_time

In [19]:
def get_deltas(telemetry, telemetry_compared_to):
    deltas = [] 

    RELATIVE_DISTANCE_COLUMN_INDEX = 3
    TIME_INDEX = 0

    for row in telemetry.values:
        rel_distance = row[RELATIVE_DISTANCE_COLUMN_INDEX]
        compared_time = row[TIME_INDEX]
 
        interpolated_time = get_interpolated_time_at_distance(telemetry=telemetry_compared_to, relative_distance_point=rel_distance)
        deltas.append([rel_distance, (compared_time-interpolated_time).total_seconds()])
                      
    return deltas 

In [20]:
deltas = get_deltas(LEC_TELEMETRY, VER_TELEMETRY)
deltas

TypeError: unsupported operand type(s) for *: 'float' and 'Timestamp'

In [12]:
laps['IsOutlap'] = pd.isna(laps['Sector1Time'])
laps['IsInlap'] = pd.isna(laps['Sector3Time'])